# Weather Data 

Source: Global Historical Climate Network Daily (Link: https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn)

GHCN has daily, monthly and yearly data available. The monthly data had some more metrics (computed using the basic ones eg no. of days more than 32C derived from TMAX data) than the daily data. However, I have used daily data as it had the best granularity and availability. The data was downloaded manually from the website for all countries, unzipped and then filtered. Once I figured out the way data was presented it was parsed and read into relevant columns. There are many missing values in the dataset. 

We are focusing on the 10 countries in the Horn of Africa:<br>
Sudan (SU) <br>
South Sudan (SS): No Data <br>
Eritrea (ER): Sparse Data <br>
Ethiopia (ET) <br>
Somalia (SO): No Data <br>
Uganda (UG): No data for last decade <br>
Tanzania (TZ) <br>
Rwanda (RW) <br>
Burundi (BI): No Data <br>
Kenya (KE): <br>



In [103]:
import pandas as pd
import numpy as np
import os
import sys
import datetime

You can loop over each line in each file and separate them out in relevant columns as they all seem to align somehow. If you do this for each line then each file you'll have built your dataset for the relevant countries. Then you need to somehow aggregate this daily weather data into monthly data? You also need geographical information about these stations which you should get separately and join with this. Once thats done you may be able to send this data for integration to others. 

In [102]:
def process_df_files(df_file):
    """
    Function that takes a pandas df (read from file) and processes it into relevant columns and size. 
    Removes all flag values keeping only metric values.
    """
    # Initiatilze a dataframe to return results; also two numpy arrays for string and numerical columns
    # There are 4 columns with str data and 31 columns for each day of the month with numerical data
    temp = pd.DataFrame()
    temp_str_data = np.empty((df_file.shape[0], 5), dtype=object)
    temp_num_data = np.empty((df_file.shape[0], 31))

    # Each line has 270 characters. Using the readme df_file information, we divide each line into multiple parts - 
    # stationid, year .. day31 etc. This is saved in the temporary arrays (used np arrays for efficiency)
    for i, each_line in df_file.iterrows():
        temp_str_data[i, 0] = each_line[0][:2]
        temp_str_data[i, 1] = each_line[0][:11]
        temp_str_data[i, 2] = each_line[0][11:15]
        temp_str_data[i, 3] = each_line[0][15:17]
        temp_str_data[i, 4] = each_line[0][17:21] 
        for j, str_pos in enumerate(range(21,266,8)):
            temp_num_data[i, j] = each_line[0][str_pos:str_pos+5]

    # Take each column from temporary arrays(both str and num) and make them into columns for our final dataframe
    temp['country'] = temp_str_data[:,0]
    temp['station_id'] = temp_str_data[:,1]
    temp['year'] = temp_str_data[:,2]
    temp['month'] = temp_str_data[:,3]
    temp['metric'] = temp_str_data[:,4]
    for column_no in range(temp_num_data.shape[1]):
        col_name = 'day'+str(column_no+1)
        temp[col_name] = temp_num_data[:,column_no]

    return temp

# Example:
process_df_files(pd.read_table("ghcnd_project\\ER000063021.dly", header=None))

,country,station_id,year,month,metric,day1,day2,day3,day4,day5,...,day22,day23,day24,day25,day26,day27,day28,day29,day30,day31
0,ER,ER000063021,1943,01,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ER,ER000063021,1943,02,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999.0,-9999.0,-9999.0
2,ER,ER000063021,1943,03,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ER,ER000063021,1943,04,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999.0
4,ER,ER000063021,1943,05,TMAX,230.0,220.0,220.0,190.0,230.0,...,270.0,260.0,270.0,270.0,270.0,270.0,270.0,280.0,270.0,260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,ER,ER000063021,2012,03,TAVG,182.0,183.0,175.0,219.0,-9999.0,...,-9999.0,-9999.0,182.0,210.0,-9999.0,212.0,227.0,182.0,-9999.0,201.0
2088,ER,ER000063021,2012,04,TAVG,231.0,206.0,195.0,-9999.0,-9999.0,...,191.0,198.0,167.0,220.0,242.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2089,ER,ER000063021,2012,05,TAVG,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2090,ER,ER000063021,2014,02,TAVG,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,190.0,-9999.0,-9999.0,205.0,219.0,240.0,180.0,-9999.0,-9999.0,-9999.0


In [8]:
# We take all the .dly files (for one station) and save them to the list. Then we process each file using above func
list_of_filenames = ["ghcnd_project\\"+filename for filename in os.listdir('ghcnd_project')]
list_of_dataframes = [process_df_files(pd.read_table(filename, header=None)) for filename in list_of_filenames]

In [104]:
# concatenate all the station dataframes we have saved in the above list. 113556 rows of data now.
wdf = pd.concat(list_of_dataframes)
wdf.head()

,country,station_id,year,month,metric,day1,day2,day3,day4,day5,...,day22,day23,day24,day25,day26,day27,day28,day29,day30,day31
0,ER,ER000063021,1943,01,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ER,ER000063021,1943,02,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999.0,-9999.0,-9999.0
2,ER,ER000063021,1943,03,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ER,ER000063021,1943,04,PRCP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999.0
4,ER,ER000063021,1943,05,TMAX,230.0,220.0,220.0,190.0,230.0,...,270.0,260.0,270.0,270.0,270.0,270.0,270.0,280.0,270.0,260.0


In [98]:
# Convert the year and month to datetime and then keep only the post 2008 data. 
wdf['year'] = pd.to_datetime(wdf['year'], format='%Y').dt.year
wdf['month'] = pd.to_datetime(wdf['month'], format='%m').dt.month
wdf2008 = wdf[wdf['year']>2008]
wdf2008.head()

,country,station_id,year,month,metric,day1,day2,day3,day4,day5,...,day22,day23,day24,day25,day26,day27,day28,day29,day30,day31
2024,ER,ER000063021,2010,6,TMAX,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,246.0,-9999.0
2025,ER,ER000063021,2010,6,TMIN,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,104.0,126.0,-9999.0,116.0,-9999.0,114.0,-9999.0,-9999.0
2026,ER,ER000063021,2010,6,TAVG,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,212.0,195.0,203.0,-9999.0,203.0,-9999.0,178.0,173.0,-9999.0
2027,ER,ER000063021,2010,7,TMIN,112.0,82.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2028,ER,ER000063021,2010,7,PRCP,0.0,-9999.0,-9999.0,79.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [105]:
# We have data for 53 stations across 6 countries. 
wdf2008.groupby('station_id').count().head()

,country,year,month,metric,day1,day2,day3,day4,day5,day6,...,day22,day23,day24,day25,day26,day27,day28,day29,day30,day31
station_id,,,,,,,,,,,,,,,,,,,,,
ER000063021,68,68,68,68,68,68,68,68,68,68,...,68,68,68,68,68,68,68,68,68,68
ET000063331,263,263,263,263,263,263,263,263,263,263,...,263,263,263,263,263,263,263,263,263,263
ET000063333,497,497,497,497,497,497,497,497,497,497,...,497,497,497,497,497,497,497,497,497,497
ET000063402,439,439,439,439,439,439,439,439,439,439,...,439,439,439,439,439,439,439,439,439,439
ET000063403,372,372,372,372,372,372,372,372,372,372,...,372,372,372,372,372,372,372,372,372,372


In [107]:
# Read stations geodata file into df
stations_geodata = pd.read_table("ghcnd-stations.txt", header=None)
stations_geodata

,0
0,ACW00011604 17.1167 -61.7833 10.1 ST JO...
1,ACW00011647 17.1333 -61.7833 19.2 ST JO...
2,AE000041196 25.3330 55.5170 34.0 SHARJ...
3,AEM00041194 25.2550 55.3640 10.4 DUBAI...
4,AEM00041217 24.4330 54.6510 26.8 ABU D...
...,...
115077,ZI000067969 -21.0500 29.3670 861.0 WEST ...
115078,ZI000067975 -20.0670 30.8670 1095.0 MASVI...
115079,ZI000067977 -21.0170 31.5830 430.0 BUFFA...
115080,ZI000067983 -20.2000 32.6160 1132.0 CHIPI...


In [108]:
def process_geodata(df_file):
    """
    Function that takes a pandas df (read from file) and processes it into relevant columns and size.
    We use column vectors in numpy to append to the final df as appending rows to dataframes is expensive. 
    """
    # Initiatilze a dataframe to return results; also two numpy arrays for string and numerical columns. There are 
    # 3 columns with str data (initialized as object as str len is not fixed) and 3 columns for numerical data.
    temp = pd.DataFrame()
    temp_str_data = np.empty((df_file.shape[0], 3), dtype=object)
    temp_num_data = np.empty((df_file.shape[0], 3))

    # Each line has 85 characters. Using the readme df_file information, we divide each line into multiple parts - 
    # id, lat, long etc. This is saved in the temporary arrays (used np arrays for efficiency)
    for i, each_line in df_file.iterrows():
        temp_str_data[i, 0] = each_line[0][:2]
        temp_str_data[i, 1] = each_line[0][:11]
        temp_str_data[i, 2] = each_line[0][41:71] 
        temp_num_data[i, 0] = each_line[0][12:20]
        temp_num_data[i, 1] = each_line[0][21:29]
        temp_num_data[i, 2] = each_line[0][31:37] 
        
    # Take each column from temporary arrays(both str and num) and make them into columns for our final dataframe
    temp['country'] = temp_str_data[:,0]
    temp['station_id'] = temp_str_data[:,1]
    temp['name'] = temp_str_data[:,2]
    temp['lat'] = temp_num_data[:,0]
    temp['long'] = temp_num_data[:,1]
    temp['elevation'] = temp_num_data[:,2]
    
    return temp

stations_geodata = process_geodata(stations_geodata)

In [109]:
# Subsetting data to keep only countries in HoA
list_of_countries = ["SU", "ER", "ET", "TZ", "RW", "KE"]
stations_geodata = stations_geodata.loc[stations_geodata["country"].isin(list_of_countries)]
stations_geodata.head()

,country,station_id,name,lat,long,elevation
33057,ER,ER000063021,ASMARA,15.283,38.917,2325.0
33058,ER,ER000063023,MASSAWA,15.617,39.450,10.0
33059,ER,ER000063043,ASSAB,13.067,42.717,14.0
33063,ET,ET000063330,MAKALE,13.500,39.483,2070.0
33064,ET,ET000063331,GONDAR,12.550,37.417,1967.0


In [110]:
# Stations geodata is merged with the weather data based on station id. Then we replace -9999 with NaN and 
# add another column for montly metric means.
wdf2008merged = wdf2008.merge(stations_geodata, how="left", on="station_id")
wdf2008merged.replace(-9999.0, np.NaN, inplace = True)
wdf2008merged["mm_mean"] = wdf2008merged.iloc[:,4:36].mean(1)
wdf2008merged.head()

,country_x,station_id,year,month,metric,day1,day2,day3,day4,day5,...,day28,day29,day30,day31,country_y,name,lat,long,elevation,mm_mean
0,ER,ER000063021,2010,6,TMAX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,246.0,NaN,ER,ASMARA,15.283,38.917,2325.0,246.000000
1,ER,ER000063021,2010,6,TMIN,NaN,NaN,NaN,NaN,NaN,...,NaN,114.0,NaN,NaN,ER,ASMARA,15.283,38.917,2325.0,120.666667
2,ER,ER000063021,2010,6,TAVG,NaN,NaN,NaN,NaN,NaN,...,NaN,178.0,173.0,NaN,ER,ASMARA,15.283,38.917,2325.0,192.692308
3,ER,ER000063021,2010,7,TMIN,112.0,82.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ER,ASMARA,15.283,38.917,2325.0,113.000000
4,ER,ER000063021,2010,7,PRCP,0.0,NaN,NaN,79.0,NaN,...,NaN,NaN,NaN,NaN,ER,ASMARA,15.283,38.917,2325.0,62.250000


## To Do:
i think the final df should have stationid, year, month and then monthly means for tmax, tmin, tavg and prcp. Thats all. We could also replicate the GHCN computed metrics such as number of days in month with temp more than 32C. Should be done by Tuesday night.